# Fill test data glosses/examples with LLMs

In [62]:
import pandas as pd
import json
import os
import openai
from dotenv import load_dotenv
import time

## 1. Load the data and compute the number of requests to make

In [65]:
FILE_TO_READ = './data/test/axolotl.test.ru.tsv'
AUGMENTED_EXAMPLES = 3

language = FILE_TO_READ.split('.')[-2]
filename = FILE_TO_READ.split('/')[-1].split('.')[0:-1]
filename = '.'.join(filename)
language, filename

('ru', 'axolotl.test.ru')

The following requets will be done:
1. For each glos-known row that doesn't have an example available, an example will be generated (+3)**
2. For each gloss-unknown row, a gloss is generated. This is also useful for achieving directly subtask 2 (+1)
3. For each gloss-known row, at least three examples are needed, so if a sense has only one example, two more are generated (+2)

** To avoid repeating requests, the generation of step 3. is also done in step 1. directly for non-example rows.

In [32]:
df = pd.read_csv(FILE_TO_READ, sep='\t')
df.info()
print("\nColumns to fill 'example':" , df['example'].isna().sum(), "(output ×3)")
print("Columns to fill 'gloss':" , df['gloss'].isna().sum(), "(output ×1)")
print("Columns to augment:" , df[~df['gloss'].isna()].shape[0]-df['example'].isna().sum(), "(output ×2)")
print("Total:", df['example'].isna().sum() + df['gloss'].isna().sum() + df[~df['gloss'].isna()].shape[0], "requests")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   usage_id              2126 non-null   object 
 1   word                  2126 non-null   object 
 2   orth                  2126 non-null   object 
 3   sense_id              424 non-null    object 
 4   gloss                 424 non-null    object 
 5   example               1990 non-null   object 
 6   indices_target_token  0 non-null      float64
 7   date                  2126 non-null   object 
 8   period                2126 non-null   object 
dtypes: float64(1), object(8)
memory usage: 149.6+ KB

Columns to fill 'example': 136 (output ×3)
Columns to fill 'gloss': 1702 (output ×1)
Columns to augment: 288 (output ×2)
Total: 2262 requests


## 2. Define methods to send and store requests to OpenAI's API

In [59]:
load_dotenv('./gpt35.env')
openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION")
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_ENDPOINT")
engine=os.getenv("OPENAI_IMPLEMENTATION")
model=os.getenv("MODEL_NAME")
model

'gpt-3.5-turbo'

In [66]:
MAX_ITERATIONS = 1 # only one repetition is allowed
def request(prompt, max_tokens=500, temperature=0, iterations=0):
    """Request a completion from the OpenAI API.
    :param prompt: The prompt to send to the API
    :param max_tokens: The maximum number of tokens in the output
    :param temperature: The degree of randomness in the output
    :return response: The response from the API
    :return usage: The count of token usage from the API { "input", "output" }
    """
    try:
        response = openai.ChatCompletion.create(
            api_key=api_key,
            api_base=api_base,
            engine=engine,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[{"role": "user", "content": prompt}],
        )
        finish_reason = response["choices"][0]["finish_reason"]
        rsp = "" if finish_reason == "content_filter" else response["choices"][0]["message"]["content"]
        return rsp, { "input": response["usage"]["prompt_tokens"], "output": response["usage"]["completion_tokens"] }, finish_reason
    except Exception as e:
        response = ""
        usage = {"input": 0, "output": 0}
        if f"{e}".startswith("The response was filtered due to the prompt triggering Azure OpenAI's content management policy"):
            finish_reason = "content_filter"
        else:
            if ("Max retries exceeded with url" in f"{e}") and (iterations < MAX_ITERATIONS):
                    print("MAX_RETRIES triggered. Retrying in 60 seconds...")
                    time.sleep(60)
                    return request(prompt, max_tokens, temperature, iterations+1) # retry after 60 seconds
            finish_reason = f"ERROR [{type(e).__name__}]: {e}"
        return response, usage, finish_reason

gen_prompt = lambda prompt, *args: prompt.format(*args)

## 3. Prepare and send requests for each of the 3 cases

### 1. For each glos-known row that doesn't have an example available, an example will be generated (+1)
Additionally, in this step, the data is augmented: an extra-2 examples are generated for each request (3 in total)

In [78]:
exp_fill = df[df['example'].isna()][['word', 'orth', 'sense_id', 'gloss', 'example']]
exp_fill

,word,orth,sense_id,gloss,example
6,мёрзлый,мёрзлый,merzlyj_g74j6SCAoAc,покрытый ледяной коркой; обледенелый,NaN
15,элемент,элементъ,element_mip-D10cqu0,"хим. простое вещество, атомный состав которого...",NaN
38,трубач,трубачъ,trubach_Y8MimbC7EPc,птица псофия,NaN
44,трубач,трубачъ,trubach_wxZia0Vo9uc,"птица выпь, бугай, бык, которая звонко трубит ...",NaN
147,косарь,косарь,kosar'_jxgi1C02d0c,"тот, кто занимается косьбой, срезает косой или...",NaN
...,...,...,...,...,...
2067,ядрышко,ядрышко,jadryshko_HMwSbBK4D3Q,уменьш.-ласк. к ядро,NaN
2077,сыворотка,сыворотка,syvorotka_DjX_AZwaxJA,"лишённая жиров и белков жидкость, получающаяся...",NaN
2091,хлыст,хлыстъ,hlyst__nJvpILiulc,"гибкий, твёрдый прут, утончающийся к концу, с ...",NaN
2092,хлыст,хлыстъ,hlyst_jQ0v9rMZV44,"сплетник, сплетница",NaN


In [84]:
# original (english): 'Give me ONLY {} line-separated usage examples of the word "{}" with the following definition:\n{}'

if language == "ru":
    prompt = 'Приведите мне ТОЛЬКО {} примера употребления слова "{}" в строках со следующим определением:\n{}'
elif language == "fi":
    prompt = 'Anna minulle VAIN {} rivieroteltua esimerkkiä sanasta "{}" seuraavalla määritelmällä:\n{}'
else: # language == "de"
    prompt = 'Geben Sie mir NUR {} durch Zeilen getrennte Verwendungsbeispiele für das Wort „{}“ mit der folgenden Definition:\n{}'

for i in range(5):
    print(prompt.format(AUGMENTED_EXAMPLES, exp_fill.iloc[i, 0], exp_fill.iloc[i, 3]), end="\n\n")

Приведите мне ТОЛЬКО 3 примера употребления слова "мёрзлый" в строках со следующим определением:
покрытый ледяной коркой; обледенелый

Приведите мне ТОЛЬКО 3 примера употребления слова "элемент" в строках со следующим определением:
хим. простое вещество, атомный состав которого однороден

Приведите мне ТОЛЬКО 3 примера употребления слова "трубач" в строках со следующим определением:
птица псофия

Приведите мне ТОЛЬКО 3 примера употребления слова "трубач" в строках со следующим определением:
птица выпь, бугай, бык, которая звонко трубит в камышах

Приведите мне ТОЛЬКО 3 примера употребления слова "косарь" в строках со следующим определением:
тот, кто занимается косьбой, срезает косой или косилкой траву или хлебные злаки



### 2. For each gloss-unknown row, a gloss is generated. This is also useful for achieving directly subtask 2 (+1)

In [80]:
gls_fill = df[df['gloss'].isna()][['word', 'orth', 'sense_id', 'gloss', 'example']]
gls_fill

,word,orth,sense_id,gloss,example
1,мёрзлый,мёрзлый,NaN,NaN,Пантелей Прокофьевич сунул Григорию мёрзлую ру...
2,мёрзлый,мёрзлый,NaN,NaN,"Между домом и рельсами, за широкой мёрзлой луж..."
3,мёрзлый,мёрзлый,NaN,NaN,"— Да, холодно, должно быть. На полу мёрзлые та..."
4,мёрзлый,мёрзлый,NaN,NaN,"Когда мосты были прорваны, безоружные солдаты,..."
5,мёрзлый,мёрзлый,NaN,NaN,Мёрзлый пар валил с загнанных насмерть лошадей...
...,...,...,...,...,...
2119,чирикать,чирикать,NaN,NaN,"Канарейка, немилосердно трещавшая целый день, ..."
2121,чирикать,чирикать,NaN,NaN,"Очарованные этой, как говорится, оргией красок..."
2123,чирикать,чирикать,NaN,NaN,"— Ладно, барышни, — произнёс я, поднимаясь. — ..."
2124,чирикать,чирикать,NaN,NaN,"— Ты, я знаю, в Ленинград собрался. Мой тебе с..."


In [81]:
# original (english): 'What does "{}" mean in the following examples (ONLY return JSON with the key "definition" - detailed definition)?\n{}'

if language == "ru":
    prompt = 'Что означает «{}» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?\n{}'
elif language == "fi":
    prompt = 'Mitä «{}» tarkoittaa seuraavissa esimerkeissä (palauta VAIN JSON avaimella "definition" - monisanainen määritelmä)?\n{}'
else: # language == "de"
    prompt = 'Was bedeutet „{}“ in den folgenden Beispielen (geben sie JSON NUR mit dem Schlüssel „definition“ zurück – ausführliche Definition)?\n{}'

for i in range(5):
    print(prompt.format(gls_fill.iloc[i, 0], gls_fill.iloc[i, 4]), end="\n\n")

Что означает «мёрзлый» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
Пантелей Прокофьевич сунул Григорию мёрзлую руку, сел на край лавки, запахивая полу тулупа, обходя взглядом Аксинью, пристывшую у люльки.

Что означает «мёрзлый» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
Между домом и рельсами, за широкой мёрзлой лужей, проходила дорога, по которой возили дрова и воду.

Что означает «мёрзлый» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
— Да, холодно, должно быть. На полу мёрзлые тараканы валяются. И мыши тоже помёрзли.

Что означает «мёрзлый» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
Когда мосты были прорваны, безоружные солдаты, московские жители, женщины с детьми, бывшие в обозе французов, — всё под влиянием силы инерции не сдавалось, а бежало вперед в лодки, в мёрзлую воду.

Что 

### 3. For each gloss-known row, two more examples will be generated (data augmentation) (+2)

In [85]:
aug_fill = df[~df['gloss'].isna()][['word', 'orth', 'sense_id', 'gloss', 'example']]
# then must check for each sense_id, how many examples are. Then, augment to at least AUGMENTED_EXAMPLES
aug_fill

,word,orth,sense_id,gloss,example
0,мёрзлый,мёрзлый,merzlyj_khrrgztit4U,"разг., устар. очень холодный; ледяной","Мерзлая земля скажется, только руку приложи От..."
6,мёрзлый,мёрзлый,merzlyj_g74j6SCAoAc,покрытый ледяной коркой; обледенелый,NaN
15,элемент,элементъ,element_mip-D10cqu0,"хим. простое вещество, атомный состав которого...",NaN
30,кабак,кабакъ,kabak_hO7-f2Xxaic,"перен., разг. место, заведение, где продают и ...",Кабаком называли собст. откупное питейное заве...
38,трубач,трубачъ,trubach_Y8MimbC7EPc,птица псофия,NaN
...,...,...,...,...,...
2099,хлыст,хлыстъ,hlyst_0S-PQCiOfWU,"спец. ствол дерева, отделённый от комля и очищ...",Хлыст в 15 сажен.
2100,слыхать,слыхать,slyhat'_UUkfjifb7-I,"более относится к вестям, слухам, что слышат, ...",Что слыхать?
2117,скорлупа,скорлупа,skorlupa_Oq5fheu46Q0,"твёрдый наружный слой яиц, орехов и т. п.; твё...","Скорлупа ореховая, миндальная. Рачья скролупа...."
2120,чирикать,чирикать,chirikat'_h05Xmk8q6z4,"издавать высокие частые звуки, характерные для...",Воробьи ватажатся и чирикают на дождь.


In [87]:
# original (english): 'Give me ONLY {} line-separated usage examples of the word "{}" with the following definition:\n{}'

if language == "ru":
    prompt = 'Приведите мне ТОЛЬКО {} примера употребления слова "{}" в строках со следующим определением:\n{}'
elif language == "fi":
    prompt = 'Anna minulle VAIN {} rivieroteltua esimerkkiä sanasta "{}" seuraavalla määritelmällä:\n{}'
else: # language == "de"
    prompt = 'Geben Sie mir NUR {} durch Zeilen getrennte Verwendungsbeispiele für das Wort „{}“ mit der folgenden Definition:\n{}'

for i in range(5):
    print(prompt.format(AUGMENTED_EXAMPLES, aug_fill.iloc[i, 0], aug_fill.iloc[i, 3]), end="\n\n")

Приведите мне ТОЛЬКО 3 примера употребления слова "мёрзлый" в строках со следующим определением:
разг., устар. очень холодный; ледяной

Приведите мне ТОЛЬКО 3 примера употребления слова "мёрзлый" в строках со следующим определением:
покрытый ледяной коркой; обледенелый

Приведите мне ТОЛЬКО 3 примера употребления слова "элемент" в строках со следующим определением:
хим. простое вещество, атомный состав которого однороден

Приведите мне ТОЛЬКО 3 примера употребления слова "кабак" в строках со следующим определением:
перен., разг. место, заведение, где продают и распивают спиртные напитки; бар, ресторан, пивная, рюмочная и т. п.

Приведите мне ТОЛЬКО 3 примера употребления слова "трубач" в строках со следующим определением:
птица псофия

